In [ ]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
import rustworkx as rx

from qiskit_nature.problems.second_quantization.lattice import Lattice
import sys
import os
import time

from heisenberg_model import HeisenbergModel
from qiskit_nature.mappers.second_quantization import LogarithmicMapper
from qiskit.algorithms import NumPyEigensolver
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram

import dask
from distributed import Client
import dask.array as da

from kagome_custom import custom_VQE

from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SPSA


from scipy.optimize import differential_evolution
import toolz
from multiprocessing import Pool

from ipywidgets import IntProgress
from IPython.display import display
import time


In [ ]:
# path_to_json = './scheduler_info.json' #CAMBIAR
# client = Client(scheduler_file = path_to_json)
# client = Client(n_workers=12)
# client

In [ ]:
if __name__ == '__main__':
    t = 1.0
    edge_list = [
        (1, 2, t),
        (2, 3, t),
        (3, 5, t),
        (5, 8, t),
        (8, 11, t),
        (11, 0, t),
        (0, 6, t),
        (6, 9, t),
        (9, 10, t),
        (10, 7, t),
        (7, 4, t),
        (4, 1, t),
        (4, 2, t),
        (2, 5, t),
        (5, 11, t),
        (11, 6, t),
        (6, 10, t),
        (10, 4, t),
    ]


    num_qubits = 12
    layers = 1
    shots = 100


    
    kagome = custom_VQE(edge_list, num_qubits, layers, shots)
    kagome.get_backend()

    

    


In [ ]:

def objective(x):
    global kagome
    job = kagome.estimator.run([kagome.qc],[kagome.ham],[x])
    energy = job.result().values[0]
    #print(energy)
    return energy
    

In [ ]:
def callback(xk,convergence):#,convergence
    global f
    f.value += 1
    #ek = objective(xk)

    #print(ek)
#     file = open('de_statevector_data3.txt','a')
#     ek = objective(xk)
#     file.write(str(ek))
#     file.write('\n')
#     file.close()


In [ ]:
# optimizerSPSA = SPSA()
# x0=np.pi*(2*np.random.rand(kagome.num_param)-1)*0
# res = optimizerSPSA.minimize(objective,x0=x0)
# print('------------')
# print(res.nfev)
# print(res.fun)
# print(res.x)


In [ ]:
max_count = 25000

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar





bounds = []
initial_point = []
for k in range(3*layers):
    bounds.append((-np.pi,np.pi))
    initial_point.append(0)
maxiter=25000
popsize=1
tol=1.e-15
pool = Pool(12)
parameters={'maxiter':maxiter,
           'popsize':popsize,
           'workers':pool.map,
           'polish':False,
           'strategy':'best1exp',
           'updating':'deferred'}


#,callback=callback
optimizerDE = differential_evolution(func=objective,bounds=bounds,callback=callback,x0=initial_point,disp=True,tol=tol,**parameters)
value = objective(optimizerDE.x)

In [ ]:

# file = open('de_statevector_data3.txt','w')
# file.close()
maxit = 50
bounds = []
initial_point = []
for k in range(3*layers):
    bounds.append((-np.pi,np.pi))
    initial_point.append(0)


for i in range(maxit):

   
    
    maxiter=25000
    popsize=10
    tol=1.e-15
    pool = Pool(60)
    parameters={'maxiter':maxiter,
               'popsize':popsize,
               'workers':pool.map,
               'polish':False,
               'strategy':'best1exp',
               'updating':'deferred'}


    #,callback=callback
    optimizerDE = differential_evolution(func=objective,bounds=bounds,x0=initial_point,disp=True,tol=tol,**parameters)
    value = objective(optimizerDE.x)
    print('----------------------------------------------')
    print(str(i)+': '+str(value))
    print(optimizerDE.x)
    print('----------------------------------------------')

    if value < -12.0:
        print('Hell yeah')
        break
    

In [ ]:
from qiskit.algorithms.optimizers import SLSQP
initial_point = np.random.rand(3)*1.e-2
optimizer = SLSQP(maxiter=500,disp=True)
values=optimizer.minimize(fun=objective,x0=initial_point)
